In [1]:
import os 
os.getcwd()

'd:\\End-to-End chicken-diesase-implementation\\research'

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [4]:
from src.cnnClassifier.constant import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

d:\End-to-End chicken-diesase-implementation


In [5]:
class ConfigurationManager:
    
    def __init__(self, configfile = CONFIG_FILE_PATH, paramsfile = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(configfile)
        self.params = read_yaml(paramsfile)
        
        create_directories([self.config.artifacts_root])
        
        
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        
        config = self.config.prepare_callbacks
        
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callbacks_config = PrepareCallbacksConfig(
            
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            
        )
        
        return prepare_callbacks_config
    

In [6]:
import time
import tensorflow as tf
from src.cnnClassifier import logger

In [7]:

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig) -> None:
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timeStamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_log_at_{timeStamp}",
            
        ) 
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        
        return tf.keras.callbacks.ModelCheckpoint(
            self.config.checkpoint_model_filepath,
            save_best_only=True
        )
        
        
    def _get_tb_ckpt_callbacks(self):
        
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
        
    

In [8]:
try:
    config = ConfigurationManager()
    get_prepare_callback_config = config.get_prepare_callbacks_config()
    Prepare_callbacks = PrepareCallback(get_prepare_callback_config)
    call_backs_list = Prepare_callbacks._get_tb_ckpt_callbacks()
    
    
except Exception as e:
    logger.exception(e)
    raise e

[2023-07-30 22:49:58,193 - INFO - common - yaml file: config\config.yaml loaded successfully]
[2023-07-30 22:49:58,201 - INFO - common - yaml file: params.yaml loaded successfully]
[2023-07-30 22:49:58,209 - INFO - common - Directory created ! artifacts]
[2023-07-30 22:49:58,209 - INFO - common - Directory created ! artifacts\prepare_callbacks\checkpoint_root_log_dir]
[2023-07-30 22:49:58,209 - INFO - common - Directory created ! artifacts\prepare_callbacks\tensorboard_root_log_dir]
